In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
from sklearn.neighbors import NearestNeighbors
properati = pd.read_csv('datos/properati_2.csv',index_col=0,error_bad_lines=False)

In [140]:
def function(x):
    if(x<1500):
        return 'green'
    if(x<2000):
        return 'pink'
    if(x<2500):
        return 'gray'
    if(x<3000):
        return 'red'
    if(x<3500):
        return 'blue'
    return 'black'
properati_CABA=properati[properati['place_name'].str.contains('Capital Federal') & properati['property_type'].str.contains('apartment')]
properati_CABA=properati_CABA[['lat-lon','price_usd_per_m2','surface_covered_in_m2']].dropna()
properati_CABA['lon']=properati_CABA['lat-lon'].str.split(",").str[0].str.replace("-34.","").astype(int)
properati_CABA['lat']=properati_CABA['lat-lon'].str.split(",").str[1].str.replace("-58.","").astype(int)
properati_CABA['price_usd_per_m2']=properati_CABA['price_usd_per_m2'].astype(int)
properati_CABA['surface_covered_in_m2']=properati_CABA['surface_covered_in_m2'].astype(int)
properati_CABA['color']=properati_CABA['price_usd_per_m2'].apply(function)

In [141]:

threedee = plt.figure().gca(projection='3d')
threedee.scatter(properati_CABA['lat'], properati_CABA['lon'], properati_CABA['surface_covered_in_m2'],c=properati_CABA['color'],marker='^')
plt.show()

ValueError: Unknown projection '3d'

# =======================================================

# IMPLEMENTACIÓN KNN


In [ ]:
properati_reduce=properati[['surface_total_in_m2','floor','rooms','price_usd_per_m2']]
properati_reduce['lon']=properati['lat-lon'].str.split(",").str[0]
properati_reduce['lat']=properati['lat-lon'].str.split(",").str[1]
properati_reduce=properati_reduce.dropna().reset_index()

In [133]:
training_samples=[]
test_samples=[]
k=10
test_samples_index=[100,500,600,700,800,900,1000,1200,1300] #Indices de los elementos de prueba en el df
for index, row in properati_reduce.iterrows():
    if (index not in test_samples_index): #Uso para entrenar todos los indices, menos los de prueba.
        training_samples.append([row['surface_total_in_m2'],row['floor'],row['rooms'],row['lat'],row['lon']])
    else:
        test_samples.append([row['surface_total_in_m2'],row['floor'],row['rooms'],row['lat'],row['lon']])

In [134]:
nbrs=NearestNeighbors(n_neighbors=k,algorithm ='ball_tree').fit(training_samples)

In [135]:
distances,indices=nbrs.kneighbors(test_samples) # Le paso las caracteristicas de los test de prueba para aplicar KNN

In [136]:
indices

array([[  96,   93,   94,   95,   98,   97,   92,  101,   99,  100],
       [ 502,  501,  504, 1385, 1370,  499,  503,  505,  500,  584],
       [1380,  242,  598, 1018,  819,  491,  605,  377,  918,  387],
       [ 693,  697,  694,  698,  695,  696,  699,  410,  233,  719],
       [ 245, 1043,  620,  845, 1322,  723, 1245,  450, 1318, 1198],
       [ 883,  896,  895,  882, 1211, 1203,  787,  802, 1093, 1099],
       [  33,  385,  193, 1367, 1369, 1368, 1366, 1364, 1365,  227],
       [ 701,  702,  700,  703, 1194,  287, 1313, 1312,  312, 1409],
       [ 849, 1408,  562,  157,  158, 1410,  539,  617,  448,  603]])

In [137]:
i=0
for index in test_samples_index:
    mean=[]
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Original : ",properati_reduce[index:index+1]['price_usd_per_m2'].iloc[0])
    for j in range(k):
        print("|","Neighbor",j,": ",properati_reduce[indices[i][j]:indices[i][j]+1]['price_usd_per_m2'].iloc[0])
        mean.append(properati_reduce[indices[i][j]:indices[i][j]+1]['price_usd_per_m2'].iloc[0])
    print("Mean: ",sum(mean)//len(mean))
    i+=1

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original :  1888.88888889
| Neighbor 0 :  1888.88888889
| Neighbor 1 :  1888.88888889
| Neighbor 2 :  1888.88888889
| Neighbor 3 :  1888.88888889
| Neighbor 4 :  1888.88888889
| Neighbor 5 :  1888.88888889
| Neighbor 6 :  1888.88888889
| Neighbor 7 :  1888.88888889
| Neighbor 8 :  1888.88888889
| Neighbor 9 :  1888.88888889
Mean:  1888.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original :  2039.47368421
| Neighbor 0 :  2039.47368421
| Neighbor 1 :  2039.47368421
| Neighbor 2 :  2039.47368421
| Neighbor 3 :  2541.66666667
| Neighbor 4 :  1971.15384615
| Neighbor 5 :  2428.57142857
| Neighbor 6 :  2039.47368421
| Neighbor 7 :  2039.47368421
| Neighbor 8 :  2039.47368421
| Neighbor 9 :  2450.0
Mean:  2162.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original :  1323.52941176
| Neighbor 0 :  2155.55555556
| Neighbor 1 :  1236.84210526
| Neighbor 2 :  1666.66666667
| Neighbor 3 :  3840.57971014
| Neighbor 4 :  1742.26804124
| Neighbor 5 :  2794.11764706
| Ne